In [4]:
import pickle
from pathlib import Path

import wget
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

In [5]:
def load_transform(file_path: str) -> pd.DataFrame:
    df = pd.read_parquet(file_path)

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df["PU_DO"] = df["PULocationID"].astype(str) + "_" + df["DOLocationID"].astype(str)

    return df

# Download the data

In [6]:
DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)

MODEL_DIR = Path("./models")
MODEL_DIR.mkdir(exist_ok=True)

TRAIN_DATA = DATA_DIR / "green_tripdata_2021-01.csv.gz"
TEST_DATA = DATA_DIR / "green_tripdata_2021-02.csv.gz"

if not TRAIN_DATA.exists():
    wget.download(
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet",
        str(TRAIN_DATA),
    )
if not TEST_DATA.exists():
    wget.download(
        "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet",
        str(TEST_DATA),
    )

In [7]:
df_train = load_transform(TRAIN_DATA)
df_test = load_transform(TEST_DATA)

In [8]:
target = "duration"
categorical_cols = ["PU_DO"]
numerical_cols = ["trip_distance"]
used_cols = categorical_cols + numerical_cols

dv = DictVectorizer()

X_train = dv.fit_transform(df_train[used_cols].to_dict(orient="records"))
y_train = df_train[target]

X_val = dv.transform(df_test[used_cols].to_dict(orient="records"))
y_val = df_test[target]

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [18]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [19]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", str(TRAIN_DATA))
    mlflow.log_param("valid-data-path", str(TEST_DATA))

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import xgboost as xgb

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [25]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[14:34:36] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.15535                          
[1]	validation-rmse:17.38054                          
[2]	validation-rmse:15.83719                          
[3]	validation-rmse:14.48597                          
[4]	validation-rmse:13.32327                          
[5]	validation-rmse:12.31041                          
[6]	validation-rmse:11.43851                          
[7]	validation-rmse:10.69552                          
[8]	validation-rmse:10.05666                          
[9]	validation-rmse:9.50962                           
[10]	validation-rmse:9.05629                          
[11]	validation-rmse:8.66757                          
[12]	validation-rmse:8.34723                          
[13]	validation-rmse:8.07295                          
[14]	validation-

In [11]:
tracking_uri = "ec2-user@ec2-18-195-53-14.eu-central-1.compute.amazonaws.com"

In [13]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("iris-experiment")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

In [14]:
trackgin_uri

'ec2-user@ec2-18-195-53-14.eu-central-1.compute.amazonaws.com'